<a href="https://colab.research.google.com/github/jiruneko/New_RAG/blob/main/New_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
import requests
import io
from urllib.parse import urlparse
from PyPDF2 import PdfReader, PdfWriter

def get_pdf_reader(input_sourse):
  base_filename = "output"
  response = requests.get(input_sourse, stream=True, timeout=30)
  response.raise_for_status()

  # URLパスからファイル名を取得
  parsed_url = urlparse(input_sourse)
  path_part = os.path.basename(parsed_url.path)
  if path_part and '.' in path_part:
    base_filename = os.path.splitext(path_part)[0]

  # コンテンツをメモリに読み込む
  pdf_content = io.BytesIO(response.content)
  reader = PdfReader(pdf_content)
  total_pages = len(reader.pages)
  return reader, base_filename, total_pages

In [ ]:
def split_pdf(input_sourse, output_dir, pages_per_chunk):
  reader, base_filename, total_pages = get_pdf_reader(input_sourse)

  if reader is None:
    print("PDFリーダーの取得に失敗しました。分割を中止しています。")
    return

  try:
    # 出力ディレクトリが存在しない場合は作成します
    os.makedirs(output_dir, exist_ok=True)
    print(f"出力ディレクトリ '{output_dir}' が確保されました。")

    # チャンクの数を計算します
    num_chunks = (total_pages + pages_per_chunk - 1)
    print(f"Splitting into {num_chunks} chunks of max {pages_per_chunk} pages each.")

    # 各チャンクを処理します
    for i in range(num_chunks):
      writer = PdfWriter()
      start_page = i * pages_per_chunk
      end_page = min(start_page + pages_per_chunk, total_pages)

      print(f"Processing chunk {i + 1}/{num_chunks} (pages {start_page + 1} - {end_page})...")

      for page_num in range(start_page, end_page):
        writer.add_page(reader.pages[page_num])

      output_filename = os.path.join(output_dir, f"{base_filename}_chunk_{i+1}.pdf")

      with open(output_filename, 'wb') as outfile:
        writer.write(outfile)
      print(f"Chunk {i+1} saved as '{output_filename}'")

    print("\nPDF splitting completed successfully!")
  except Exception as e:
    print(f"An error occurred during the splitting process: {e}")